# Analyze the data

Time to design the crevasse detection algorithm.

## Eye-ball

## Approach that mimics DDA-ice

## Visualization

In [1]:
%matplotlib widget

In [53]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde

In [3]:
filename = 'download/ATL03_20190805232841_05940403_004_01.h5'

In [37]:
with h5py.File(filename, 'r') as f:
    h_ph = f['gt1l']['heights']['h_ph'][:]
    lat_ph = f['gt1l']['heights']['lat_ph'][:]
    lon_ph = f['gt1l']['heights']['lon_ph'][:]
    dist_ph = f['gt1l']['heights']['dist_ph_along'][:]
    geo_segment_length = f['gt1l']['geolocation']['segment_length'][:]
    geo_segment_ph_count = f['gt1l']['geolocation']['segment_ph_cnt'][:]
    quality_ph = f['gt1l']['heights']['signal_conf_ph'][:]

In [5]:
def make_dist(ph_count, seg_length, dist_ph):
    repeat_num = np.cumsum(seg_length) - seg_length[0]
    final_dist = np.repeat(repeat_num, ph_count)
    final_dist += dist_ph
    return final_dist

In [8]:
fin_dist_ph = make_dist(geo_segment_ph_count, geo_segment_length, dist_ph)

In [45]:
idx = np.logical_and(fin_dist_ph > 2153000, fin_dist_ph < 2154000)

x = fin_dist_ph[idx]
y = h_ph[idx]
q = quality_ph[idx, 0]

In [46]:
q

array([0, 0, 4, ..., 4, 0, 0], dtype=int8)

In [52]:
fig, ax = plt.subplots(1, 1, figsize=(7, 3))

ax.plot(x[q == 0], y[q == 0], '.', markersize=1, label='noise')
ax.plot(x[q == 2], y[q == 2], '.', markersize=1, label='low')
ax.plot(x[q == 3], y[q == 3], '.', markersize=1, label='mid')
ax.plot(x[q == 4], y[q == 4], '.', markersize=1, label='high')
ax.set_xlim(2153000, 2154000)
ax.set_ylim(320, 360)
ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [139]:
xy = np.vstack([x, y])
z = gaussian_kde(xy, bw_method=0.01)(xy)
thres = 0.00004

fig, ax2 = plt.subplots(1, 1, figsize=(7, 3))

pt_style = {'s': 5, 'edgecolor': None}
ax2.scatter(x, y, c=z, **pt_style)
ax2.plot(x[z > thres], y[z > thres], '.', color='xkcd:red', markersize=1)
ax2.set_xlim(2153000, 2154000)
ax2.set_ylim(320, 360)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(320.0, 360.0)

In [56]:
import seaborn

In [63]:
fig, ax3 = plt.subplots(1, 1, figsize=(7, 3))

seaborn.kdeplot(x=x, y=y, ax=ax3)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:>

In [66]:
from scipy.interpolate import UnivariateSpline

xh = x[q == 4]
yh = y[q == 4]

In [69]:
idx2 = np.argsort(xh)

xhh = xh[idx2]
yhh = yh[idx2]

In [94]:
spl = UnivariateSpline(xhh, yhh, s=0)
xs = np.linspace(2153000, 2154000, 501)
ys = spl(xs)

ValueError: x must be strictly increasing if s = 0

In [92]:
ys

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, na

In [93]:
fig, ax4 = plt.subplots(1, 1, figsize=(7, 3))

ax4.plot(xhh, yhh, '.', markersize=1, label='high')
# ax4.plot(xs, ys)
ax4.set_xlim(2153000, 2154000)
ax4.set_ylim(320, 360)
ax4.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [107]:
fig, ax5 = plt.subplots(1, 1, figsize=(7, 3))

seaborn.kdeplot(x=xhh, y=yhh, ax=ax5, bw_method=0.01, fill=True)
ax5.plot(xhh, yhh, '.', markersize=1, color='xkcd:red')
ax5.set_xlim(2153000, 2154000)
ax5.set_ylim(320, 360)
# ax5.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(320.0, 360.0)

In [137]:
xyhh = np.vstack([xhh, yhh])
zhh = gaussian_kde(xyhh, bw_method=0.02)(xyhh)
thres = 0.0005

fig, ax6 = plt.subplots(1, 1, figsize=(7, 3))

pt_style = {'s': 5, 'edgecolor': None}
ax6.scatter(xhh, yhh, c=zhh, **pt_style)
ax6.plot(xhh[zhh > thres], yhh[zhh > thres], '.', color='xkcd:red', markersize=1)
ax6.set_xlim(2153000, 2154000)
ax6.set_ylim(320, 360)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(320.0, 360.0)

8.068187841501603e-05

In [79]:
from scipy.signal import find_peaks

In [ ]:
peaks, properties = find_peaks(x, prominence=(None, 0.6))
properties["prominences"].max()
0.5049999999999999
plt.plot(x)
plt.plot(peaks, x[peaks], "x")
plt.show()